## import library

In [8]:
import osmnx as ox
from geopy.distance import geodesic
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from tqdm import tqdm
import re
import time
import requests

## EDA

In [3]:
# 서초구 개별 주정차단속 현황
illegal_info = pd.read_csv("../../../Data/2201-2312개별서초구주정차단속현황.csv", encoding='cp949')

print("컬럼 개수 : {}".format(len(illegal_info)))

컬럼 개수 : 279041


In [ ]:
# 위반 내용 비율

illegal_info['위반내용'].value_counts(dropna=True, normalize=True)

위반내용
주정차금지(황색실선)구역    0.664791
보도               0.123437
주차금지(황색점선)구역     0.097115
도로 모퉁이           0.035350
횡단보도             0.027440
소화전              0.011690
교통소통장애           0.010522
교차로              0.010253
안전지대             0.008633
버스정류소            0.006709
주차방법위반           0.002426
다리               0.000530
주차구획선외 주차        0.000477
특별구역             0.000265
소방차(긴급차량)통행장애    0.000111
이중주차             0.000090
소방기계 비치장소        0.000086
건널목              0.000036
터널               0.000029
이면도로주차           0.000011
Name: proportion, dtype: float64

# Data Preprocessing

> 서초구 개별 주정차 단속 현황에서 CCTV 기반단속 (CCTV 정보 코드; e.g., N00501) 건수와 일반 단속 건수 분리

> CCTV 기반 단속 건수 처리
- CCTV 기반 단속 건수에 대해서는 안심이 CCTV 연계 현황과 매칭하여 단속 위치 위/경도 추출

> 일반 단속 건수 처리
- 단속동/단속장소 기반 패턴 처리 후 지오코딩으로 위/경도 추출
- 지오코딩 안된 관측치는 제거 (143558 -> 140855)

In [2]:
# 서초구 개별 주정차단속 현황
illegal_info = pd.read_csv("../../../Data/2201-2312개별서초구주정차단속현황.csv", encoding='cp949')
display(illegal_info.head(3))

# 안심이 CCTV 정보
df_ansimi = pd.read_csv("../../../Data/서울시 서초구 안심이 CCTV 연계 현황.csv", encoding='cp949')
display(df_ansimi.head(3))

,단속일시,과태료 원금,단속동,단속장소,위반내용
0,2022-01-01 00:02,40000,서초동,서울특별시 서초구 반포대로22길 73 (서초동),주정차금지(황색실선)구역
1,2022-01-01 00:03,40000,서초동,서울특별시 서초구 반포대로22길 73 (서초동),주정차금지(황색실선)구역
2,2022-01-01 00:11,40000,방배동,서울특별시 서초구 효령로25길 59 (방배동),주정차금지(황색실선)구역


,자치구,안심 주소,위도,경도,CCTV 수량,수정 일시
0,서초구,A13002;1/1rxxxx3;서초3동 1488-4;상명달어린이공원 시소앞,37.4822,127.0057,1,2022-12-01
1,서초구,A13003;1/1rxxxx3;서초3동 1518-1;하명달어린이공원 미끄럼틀앞,37.4859,127.0055,1,2022-12-01
2,서초구,A13004;1/1rlxxx3;반포1동 720;언구비어린이공원 그네앞,37.5090,127.0202,1,2022-12-01


In [ ]:
def split_and_process_cctv_base_df(df, df_ansimi, save_dir=None):
    # data split

    mask_cctv_code = df['단속장소'].str.contains(r"^([A-Z]{1,2}\d{5})", regex=True, na=False) # cctv 정보 있는것, 없는것 분리
    df_cctv = df[mask_cctv_code].copy()

    ## preprocess
    df_cctv['코드'] = df_cctv['단속장소'].str.extract(r"^([A-Z]{1,2}\d{5})")
    df_cctv['단속위치_original'] = df_cctv['단속장소'].str.extract(r"^[A-Z]{1,2}\d{5}\s*(.*)")

    df_ansimi['코드'] = df_ansimi['안심 주소'].str.extract(r"^([A-Z]{1,2}\d{5})") # cctv 코드 추출
    df_ansimi['지번'] = df_ansimi['안심 주소'].str.extract(r"^[^;]+;[^;]+;([^;]+)") # 지번 추출
    df_ansimi['위치정보'] = df_ansimi['안심 주소'].str.extract(r";([^;]+)$") # 위치 

    df_cctv_merged = pd.merge(df_cctv, df_ansimi, left_on='코드', right_on='코드', how='left')

    # feature generation
    df_cctv_merged['단속장소_new'] = df_cctv_merged['지번'] + " " + df_cctv_merged['위치정보']

    df_new = df_cctv_merged[['단속일시', '과태료 원금', '단속동', '단속장소_new', '코드', '지번', '위치정보', '위반내용', '위도', '경도']].drop_duplicates()
    df_new = df_new.reset_index().drop(columns='index', axis=1)
    if save_dir:
        df_new.to_csv(save_dir, encoding='cp949', index=False)
        
    return df_new

In [10]:
df_cctv_illegal_info = split_and_process_cctv_base_df(illegal_info, df_ansimi, save_dir="../../../Data/서초구주정차단속및CCTV정제데이터/CCTV기반단속개별건수.csv")
display(df_cctv_illegal_info.head(5))
print(len(df_cctv_illegal_info))

C:\Users\chobh\AppData\Local\Temp\ipykernel_27896\1610876321.py:4: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask_cctv_code = df['단속장소'].str.contains(r"^([A-Z]{1,2}\d{5})", regex=True, na=False) # cctv 정보 있는것, 없는것 분리


,단속일시,과태료 원금,단속동,단속장소_new,코드,지번,위치정보,위반내용,위도,경도
0,2022-01-01 08:00,40000,방배4동,방배4동 876-1 세븐일레븐 앞,P21040,방배4동 876-1,세븐일레븐 앞,주차금지(황색점선)구역,37.4892,126.9936
1,2022-01-01 08:02,40000,서초3동,서초3동 1715-10 남촌빌딩 주변,N05010,서초3동 1715-10,남촌빌딩 주변,주차금지(황색점선)구역,37.4934,127.0124
2,2022-01-01 08:02,40000,서초3동,서초3동 1715-10 남촌빌딩 주변,N05010,서초3동 1715-10,남촌빌딩 주변,주차금지(황색점선)구역,37.4934,127.0125
3,2022-01-01 08:02,40000,서초1동,서초1동 1625-7 프리우스주변,P12002,서초1동 1625-7,프리우스주변,주차금지(황색점선)구역,37.4878,127.0163
4,2022-01-01 08:02,40000,서초4동,서초4동 1689-6 정관장 앞,N21031,서초4동 1689-6,정관장 앞,주정차금지(황색실선)구역,37.4955,127.0189


163893


In [45]:
mask_cctv_code = illegal_info['단속장소'].str.contains(r"^([A-Z]{1,2}\d{5})", regex=True, na=False)
df_new = illegal_info[~mask_cctv_code].copy()
display(df_new.head(5))
print(len(df_new))

C:\Users\chobh\AppData\Local\Temp\ipykernel_1360\3267912493.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask_cctv_code = illegal_info['단속장소'].str.contains(r"^([A-Z]{1,2}\d{5})", regex=True, na=False)


,단속일시,과태료 원금,단속동,단속장소,위반내용
0,2022-01-01 00:02,40000,서초동,서울특별시 서초구 반포대로22길 73 (서초동),주정차금지(황색실선)구역
1,2022-01-01 00:03,40000,서초동,서울특별시 서초구 반포대로22길 73 (서초동),주정차금지(황색실선)구역
2,2022-01-01 00:11,40000,방배동,서울특별시 서초구 효령로25길 59 (방배동),주정차금지(황색실선)구역
3,2022-01-01 05:40,40000,방배동,서울특별시 서초구 방배천로 155,교통소통장애
4,2022-01-01 05:40,40000,방배동,서울특별시 서초구 방배천로 155,교통소통장애


143558


In [4]:
# 전체 패턴 개수 확인

print((sum(df_new['단속동'].str.endswith('동')) + sum(df_new['단속동'].str.endswith('길')) + sum(df_new['단속동'].str.endswith('로'))) == len(df_new))

## 단속동 / 단속장소 패턴 확인
# 패턴 1 : 서초동, 방배동
display(df_new[df_new['단속동']=='서초동'].head(2))

# 패턴 2 : 서초4동, 방배1동
display(df_new[df_new['단속동']=='서초4동'].head(2))

# 패턴 3 : 방배로, 방배천로
display(df_new[df_new['단속동']=='방배천로'].head(2))

# 패턴 4 : 동광로7길, 사평대로55길
display(df_new[df_new['단속동']=='동광로7길'].head(2))


True


,단속일시,과태료 원금,단속동,단속장소,위반내용
0,2022-01-01 00:02,40000,서초동,서울특별시 서초구 반포대로22길 73 (서초동),주정차금지(황색실선)구역
1,2022-01-01 00:03,40000,서초동,서울특별시 서초구 반포대로22길 73 (서초동),주정차금지(황색실선)구역


,단속일시,과태료 원금,단속동,단속장소,위반내용
254444,2023-10-01 02:55,40000,서초4동,서초동 1310-26,도로 모퉁이
254477,2023-10-01 18:07,40000,서초4동,서초동 1393,횡단보도


,단속일시,과태료 원금,단속동,단속장소,위반내용
295,2022-01-02 08:39,40000,방배천로,방배동 440-17,보도
316,2022-01-02 09:57,40000,방배천로,방배동 491-1,보도


,단속일시,과태료 원금,단속동,단속장소,위반내용
9,2022-01-01 07:35,40000,동광로7길,방배동 822-23,도로 모퉁이
1406,2022-01-05 08:21,40000,동광로7길,방배동 822-23,도로 모퉁이


In [ ]:
# def format_full_address_by_pattern(row):
#     raw_addr = row['단속장소'].strip()
#     dong = row['단속동'].strip()

#     # 괄호 제거 (ex. 서울특별시 서초구 반포대로22길 73 (서초동))
#     raw_addr = re.sub(r"\s*\([^()]*\)?", "", raw_addr)


#     # 패턴 ①: ~동 (숫자 없음) + 도로명 주소
#     if re.match(r"^[가-힣]{2,4}동$", dong) and re.search(r"[가-힣]+\d+길?\s*\d+", raw_addr):
#         return f"{raw_addr}"

#     # 패턴 ②: ~x동 (숫자 포함 행정동) + 지번
#     elif re.match(r"^[가-힣]{2,4}\d{1,2}동$", dong) and re.match(r"^\d{1,5}-\d{1,5}$", raw_addr):
#         return f"서울특별시 서초구 {dong} {raw_addr}"

#     # 패턴 ③: ~로 (도로명) + 지번
#     elif re.match(r"^[가-힣]+로$", dong) and re.match(r"^\d{1,5}-\d{1,5}$", raw_addr):
#         return f"서울특별시 서초구 {raw_addr}"

#     # 패턴 ④: ~로\d+길 (도로명+세부길) + 지번
#     elif re.match(r"^[가-힣]+로\d+길$", dong) and re.match(r"^\d{1,5}-\d{1,5}$", raw_addr):
#         return f"서울특별시 서초구 {raw_addr}"

#     # 예외 fallback
#     else:
#         return f"서울특별시 서초구 {raw_addr}"
    
# def address_to_lat_lon(address, retry=3):
#     for _ in range(retry):
#         try:
#             point = ox.geocoder.geocode(address)
#             if isinstance(point, tuple):
#                 lat, lon = point
#                 return pd.Series({'위도' : lat, '경도' : lon})
#             else:
#                 return pd.Series({'위도' : None, '경도' : None})
            
#         except Exception:
#             # time.sleep(1)
#             pass
        
#     return pd.Series({'위도' : None, '경도' : None})

In [ ]:
# df_new['full_address'] = df_new.apply(format_full_address_by_pattern, axis=1)

In [ ]:
# # 단속동 컬럼 패턴
# # 패턴 1 : 서초동, 방배동
# display(df_new[df_new['단속동']=='서초동'].head(2))

# # 패턴 2 : 서초4동, 방배1동
# display(df_new[df_new['단속동']=='서초4동'].head(2))

# # 패턴 3 : 방배로, 방배천로
# display(df_new[df_new['단속동']=='방배천로'].head(2))

# # 패턴 4 : 동광로7길, 사평대로55길
# display(df_new[df_new['단속동']=='동광로7길'].head(2))


,단속일시,과태료 원금,단속동,단속장소,위반내용,full_address
0,2022-01-01 00:02,40000,서초동,서울특별시 서초구 반포대로22길 73 (서초동),주정차금지(황색실선)구역,서울특별시 서초구 반포대로22길 73
1,2022-01-01 00:03,40000,서초동,서울특별시 서초구 반포대로22길 73 (서초동),주정차금지(황색실선)구역,서울특별시 서초구 반포대로22길 73


,단속일시,과태료 원금,단속동,단속장소,위반내용,full_address
254444,2023-10-01 02:55,40000,서초4동,서초동 1310-26,도로 모퉁이,서울특별시 서초구 서초동 1310-26
254477,2023-10-01 18:07,40000,서초4동,서초동 1393,횡단보도,서울특별시 서초구 서초동 1393


,단속일시,과태료 원금,단속동,단속장소,위반내용,full_address
295,2022-01-02 08:39,40000,방배천로,방배동 440-17,보도,서울특별시 서초구 방배동 440-17
316,2022-01-02 09:57,40000,방배천로,방배동 491-1,보도,서울특별시 서초구 방배동 491-1


,단속일시,과태료 원금,단속동,단속장소,위반내용,full_address
9,2022-01-01 07:35,40000,동광로7길,방배동 822-23,도로 모퉁이,서울특별시 서초구 방배동 822-23
1406,2022-01-05 08:21,40000,동광로7길,방배동 822-23,도로 모퉁이,서울특별시 서초구 방배동 822-23


In [ ]:
def split_and_process_enforcement_base_df(df, geocoding=False, save_dir=None, api_key=None, delay=0.4):
    tqdm.pandas()

    ### 1. 주소 포맷 정제 함수 ###
    def format_full_address_by_pattern(row):
        raw_addr = row['단속장소'].strip()
        dong = row['단속동'].strip()
        raw_addr = re.sub(r"\s*\([^()]*\)?", "", raw_addr)

        if re.match(r"^[가-힣]{2,4}동$", dong) and re.search(r"[가-힣]+\d+길?\s*\d+", raw_addr):
            return f"{raw_addr}"
        elif re.match(r"^[가-힣]{2,4}\d{1,2}동$", dong) and re.match(r"^\d{1,5}-\d{1,5}$", raw_addr):
            return f"서울특별시 서초구 {dong} {raw_addr}"
        elif re.match(r"^[가-힣]+로$", dong) and re.match(r"^\d{1,5}-\d{1,5}$", raw_addr):
            return f"서울특별시 서초구 {raw_addr}"
        elif re.match(r"^[가-힣]+로\d+길$", dong) and re.match(r"^\d{1,5}-\d{1,5}$", raw_addr):
            return f"서울특별시 서초구 {raw_addr}"
        else:
            return f"서울특별시 서초구 {raw_addr}"

    ### 2. 주소 유형 판단 함수 ###
    def detect_address_type(addr: str) -> str:
        if re.search(r"(길|로|대로|거리)", addr):
            return "road"
        elif re.search(r"\d{1,5}-?\d{0,5}$", addr):
            return "parcel"
        else:
            return "road"  # fallback

    ### 3. VWorld API 기반 주소 → 위경도 변환 (캐싱 포함) ###
    geocode_cache = {}

    def geocode_vworld_smart(address: str, retry=3):
        if address in geocode_cache:
            return geocode_cache[address]

        addr_type = detect_address_type(address)
        params = {
            "service": "address",
            "request": "getcoord",
            "crs": "epsg:4326",
            "address": address,
            "format": "json",
            "type": addr_type,
            "key": api_key
        }

        for _ in range(retry):
            try:
                res = requests.get("https://api.vworld.kr/req/address", params=params)
                if res.status_code == 200:
                    data = res.json()
                    if data.get('response', {}).get('status') == 'OK':
                        point = data['response']['result']['point']
                        lat = float(point['y'])
                        lon = float(point['x'])
                        result = pd.Series({'위도': lat, '경도': lon})
                        geocode_cache[address] = result
                        time.sleep(delay)
                        return result
            except Exception:
                time.sleep(1)

        geocode_cache[address] = pd.Series({'위도': None, '경도': None})
        return geocode_cache[address]

    ### 4. 본 처리 ###
    # CCTV 코드 기반 탐지건은 제외
    mask_cctv_code = df['단속장소'].str.contains(r"^([A-Z]{1,2}\d{5})", regex=True, na=False)
    df_enforcement = df[~mask_cctv_code].copy()

    # full address 생성
    df_enforcement['full_address'] = df_enforcement.apply(format_full_address_by_pattern, axis=1)
    
    # 위/경도 na값 제거
    df_enforcement = df_enforcement.dropna(subset=['위도', '경도']).reset_index(drop=True)

    # 지오코딩 처리
    if geocoding:
        if not api_key:
            raise ValueError("지오코딩을 사용하려면 'api_key'를 반드시 지정해야 합니다.")
        df_enforcement[['위도', '경도']] = df_enforcement['full_address'].progress_apply(geocode_vworld_smart)

    # 저장
    if save_dir:
        df_enforcement.to_csv(save_dir, encoding='cp949', index=False)

    return df_enforcement

In [42]:
df_test = df_new.iloc[:10]
df_test

,단속일시,과태료 원금,단속동,단속장소,위반내용
0,2022-01-01 00:02,40000,서초동,서울특별시 서초구 반포대로22길 73 (서초동),주정차금지(황색실선)구역
1,2022-01-01 00:03,40000,서초동,서울특별시 서초구 반포대로22길 73 (서초동),주정차금지(황색실선)구역
2,2022-01-01 00:11,40000,방배동,서울특별시 서초구 효령로25길 59 (방배동),주정차금지(황색실선)구역
3,2022-01-01 05:40,40000,방배동,서울특별시 서초구 방배천로 155,교통소통장애
4,2022-01-01 05:40,40000,방배동,서울특별시 서초구 방배천로 155,교통소통장애
5,2022-01-01 05:56,40000,서초동,서울특별시 서초구 효령로 292,버스정류소
6,2022-01-01 06:00,40000,서초동,"서울특별시 서초구 서초대로77길 37 (서초동, 서초동",주정차금지(황색실선)구역
7,2022-01-01 06:02,40000,서초동,서울특별시 서초구 강남대로 433,주정차금지(황색실선)구역
8,2022-01-01 06:03,40000,서초동,서울특별시 서초구 강남대로 435,주정차금지(황색실선)구역
9,2022-01-01 07:35,40000,동광로7길,방배동 822-23,도로 모퉁이


In [48]:
API_KEY = ""
df_test = split_and_process_enforcement_base_df(df_new, geocoding=True, api_key=API_KEY, save_dir="../../../Data/서초구주정차단속및CCTV정제데이터/일반단속개별건수_geocoded.csv")

C:\Users\chobh\AppData\Local\Temp\ipykernel_1360\1975971683.py:69: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask_cctv_code = df['단속장소'].str.contains(r"^([A-Z]{1,2}\d{5})", regex=True, na=False)
100%|██████████| 143558/143558 [2:07:18<00:00, 18.79it/s]  


In [ ]:
# VWorld api test
api_url = "https://api.vworld.kr/req/address?"

params = {
	"service": "address",
	"request": "getcoord",
	"crs": "epsg:4326",
	"address": "서울특별시 서초구 반포대로22길 73 (서초동)",
	"format": "json",
	"type": "road",
	"key": API_KEY
}

response = requests.get(api_url, params=params)

if response.status_code == 200:
    print(response.json())
    
response.json()['response']['result']['point']

{'response': {'service': {'name': 'address', 'version': '2.0', 'operation': 'getcoord', 'time': '27(ms)'}, 'status': 'OK', 'input': {'type': 'road', 'address': '서울특별시 서초구 반포대로22길 73 (서초동)'}, 'refined': {'text': '서울특별시 서초구 반포대로22길 73 (서초동)', 'structure': {'level0': '대한민국', 'level1': '서울특별시', 'level2': '서초구', 'level3': '서초동', 'level4L': '반포대로22길', 'level4LC': '', 'level4A': '서초3동', 'level4AC': '1165053000', 'level5': '73', 'detail': ''}}, 'result': {'crs': 'EPSG:4326', 'point': {'x': '127.012357333', 'y': '37.490809621'}}}}


In [56]:
# 데이터 확인

null_count = df_test[df_test['위도'].isnull() | df_test['경도'].isnull()].shape[0]
null_count

2703

In [58]:
print(len(df_test))

df_test = df_test.dropna(subset=['위도', '경도']).reset_index(drop=True)

print(len(df_test))

143558
140855
